<span style = "color: red">Following code is the working version of using MPS shors for factorization of co-prime numbers</span>

- In another code called `MPS Shors Analysis` we will look at the different properties of using MPS Shors and try to compare with the other versions of Shor's available

# Installations

In [1]:
!pip install --quiet cotengra
!pip install --quiet kahypar
!pip install --quiet scikit_optimize
!pip install --quiet networkx
!pip install --quiet tqdm
!pip install --quiet quimb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.8/189.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 42.3 MB/s eta 0:00:00


# Imports

In [2]:
import numpy as np
from numpy import linalg as LA
from numpy import pi

import quimb
import quimb.tensor as qtn
import quimb as qu

import networkx as nx
import cotengra as ctg
import time
import tqdm

import math
from math import pi, ceil, log2

/usr/local/lib/python3.11/dist-packages/cotengra/hyperoptimizers/hyper.py:57: UserWarning: Couldn't find `optuna`, `cmaes`, `baytune (btb)`, `chocolate`, or `nevergrad` so will use completely random sampling in place of hyper-optimization.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/cotengra/hyperoptimizers/hyper.py:76: UserWarning: Couldn't find `optuna`, `cmaes`, `baytune (btb)`, `chocolate`, or `nevergrad` so will use completely random sampling in place of hyper-optimization.
  warnings.warn(


# Functions

In [3]:
#Variable Definitons
H = quimb.hadamard()

X = qu.pauli('X')


#Functions
def toBinary(val):
  binary = ''
  loop = val

  while loop>0:
    temp = loop % 2
    loop = loop//2
    binary += str(temp)

  binary = binary[::-1]
  return binary

#Binary to Integer
def toInteger(val: str):
  num = 0
  val = val[::-1]

  for i in range(len(val)):
    if val[i] == '0':
      continue

    else:
      num += 2**i

  return num

#The inverse QFT operation
def inverse_qft(mps_state, binary, contract_value_h, contract_value):
  n = len(binary)
  H = quimb.hadamard()
  swap = np.array(quimb.swap())

  for i in range(int(n/2)):
    mps_state.gate_(swap, (i, n-(i+1)), tags = 'Swap', max_bond = 2, contract = contract_value)

  for i in np.arange(n):
    for j in np.arange(i+1, n):
      phase = -1/2**(j-i)
      # cphase = np.array(quimb.controlled('z'))**phase
      cphase = np.array([
    [1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
    [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
    [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
    [0.+0.j, 0.+0.j, 0.+0.j, np.exp(1j * phase)]
]) #The actual Controlled Phase operation as denoted in the theory

      mps_state.gate_(cphase, (j,i), tags = 'CP', max_bond = 2, contract = contract_value)

    mps_state.gate_(H, i, tags = 'H', contract = contract_value_h)

  return mps_state


#The black box operation
def opUx(mps_p, a, power, N):
  # n = len(mps_p.site_inds)
  n = len(toBinary(t))
  swap = qu.swap()
  X = qu.pauli('X')

  for i in range(power):
    for j in range(n-1, 0, -1):
      # print("Debug value first j: ", j)
      mps_p.gate_(swap, (0, j), tags = 'Swap', max_bond = 2, contract = True)
    for j in range(n):
      # print("Debug value second j: ", j)
      mps_p.gate_(X, j, tags = 'X', max_bond = 2, contract = True)

  return mps_p

# The main code

In [33]:
# The number to be factorized
N = 50463

#The guess number
a= 37



In [34]:

e = 0.1
L = ceil(log2(N))
t = ceil(2 * L + 1 + (2 + 1/(2*e)))


n = len(toBinary(t)) #The first register qubits initialized as 0
m = 2 * n #Total qubits number in the circuit

p = qtn.MPS_computational_state('0'*m, dtype = 'float64', cyclic = False)



for i in range(n):
  p.gate_(H, i) #Applying Hadamard for the first register qubits

p.gate_(X, m-1) #Changing the last qubit of the second register to state |1>

for i in range(n):
  opUx(p, a, 2**i, N) #Applying the Controlled Unitary Black operation from the first n qubits


inverse_qft(p, toBinary(t), True, True)  # Perfoming the inverse QFT operation to get back the state containing the order

MatrixProductState(tensors=14, indices=30, L=12, max_bond=2)

In [35]:
statevector = np.abs(p.to_dense())

#The actual statevector containing the 2^n states rather than 2^m

statevector_pm=np.array([statevector[i] for i in statevector.nonzero()[0]])
statevector_pm /= np.linalg.norm(statevector_pm)

state_dict = {i : statevector_pm[i, 0] for i in range(1, 32)}

keys_quimb = [i for i in state_dict.keys()]

# Printing out the factors

In [36]:
import math
import numpy as np
from fractions import Fraction

for i in keys_quimb:
    if i % 2 == 0:
        x = (a**(i//2)) % N
        temp_p = math.gcd(x-1, N)
        temp_q = math.gcd(x+1, N)

        if temp_p != 1 and temp_p != N:
            p = temp_p
        if temp_q != 1 and temp_q != N:
            q = temp_q
        else:
            continue
    else:
        continue

if N % a == 0:
    p = a
    q = N // a
    print("The value you chose for \"a\" is a factor of \"N\"!")
    print("N =", N)
    print("Prime Factor 1 =", p)
    print("Prime Factor 2 =", q)
else:
    if p != 0:
        q = N // p
    elif q != 0:
        p = N // q

    if p == 0 and q == 0:
        print("Try a different value for \"a\"!")
    else:
        print("The number to be factorized N =", N)
        print("Prime Factor 1 =", p)
        print("Prime Factor 2 =", q)


The number to be factorized N = 50463
Prime Factor 1 = 189
Prime Factor 2 = 267
